# Data wrangling

Since the data tables on the [Footballguys](https://www.footballguys.com/) website are fully rendered in HTML, we might be able to scrape the data without too much trouble. This gives us good control over exactly what data we download and an easy mechanism by which to update it throughout the season. Let's give it a try using [urllib](https://docs.python.org/3/howto/urllib2.html) and [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/).

In [1]:
# Standard library imports
import time
import pickle
import urllib.request
from itertools import product
from random import randrange

# PyPI imports
import pandas as pd
from bs4 import BeautifulSoup

# Set the data file paths
raw_data_path='../data/raw_data_all_positions.pkl'
parsed_data_path='../data/parsed_data_all_positions.parquet'

## 1. Download and parse HTML data

The available data spans 1996 to 2024 and each year has 18 weeks of data. We also will want to download the data for multiple positions. But, let's start with just one. We also need to pick a scoring scheme, let's go with PPR. We can easily change this later. We will use a loop to construct and download the URL for each year and week and parse and collect the data as we get it.

**Note**: Downloading all of the data for one position takes just over 45 minutes.

### 1.1. Download function

In [2]:
def download_url(url: str) -> bytes:
    '''Takes string url, downloads URL and returns HTML bytes object'''

    headers={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Host": "httpbin.io",
        "Sec-Ch-Ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Linux"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "cross-site",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }

    # Create the request
    request_params = urllib.request.Request(
        url=url,
        headers=headers
    )   

    # Get the html
    with urllib.request.urlopen(request_params) as response:
        html=response.read()

    return html

### 1.2. HTML parsing function

In [3]:
def parse_html_table(html: bytes, year: int, week: int, profile: str) -> pd.DataFrame:
    '''Takes a html bytes object from URL, parses data table, adds
    year, week, position and scoring profile and returns as pandas dataframe'''

    # Extract the table rows
    soup=BeautifulSoup(html, 'html.parser')
    table=soup.find('table',{'class':'datasmall table'})
    table_rows=table.find_all('tr')

    # Get the column names from the first row
    columns=table_rows[0].find_all('th')
    column_names=[column.getText() for column in columns]
    column_names.extend(['Year', 'Week', 'Scoring profile'])

    # Get the values for each row
    data=[]

    for row in table_rows[1:]:
        columns=row.find_all('td')
        values=[column.getText() for column in columns]
        values.extend([year, week, profile])
        data.append(values)

    # Convert to pandas dataframe and return
    return pd.DataFrame(columns=column_names, data=data)

### 1.3. Main download loop

In [4]:
%%time

# Main script to download data
download_data = False  # Or False, depending on what you want to do

if download_data is True:
    positions = ['qb', 'rb', 'wr', 'te']
    profile = 'p'
    years = list(range(2020, 2024))
    weeks = list(range(1, 19))

    # Empty dict. to store dataframes for each position
    position_data={}

    # Loop on positions first, and create a separate dataframe for each
    for position in positions:

        # Empty list to collect data for this position
        results = []

        for year, week in product(years, weeks):
            print(f'Downloading {position.upper()}, {year}, week {week}', end='\r')
            url = f'https://www.footballguys.com/playerhistoricalstats?pos={position}&yr={year}&startwk={week}&stopwk={week}&profile={profile}'
            
            # Get the HTML
            html = download_url(url)
            
            # Parse the HTML
            result = parse_html_table(html, year, week, profile)
            
            # Collect the result
            results.append(result)

            # Wait before downloading the next page
            time.sleep(randrange(1, 5))

        # Combine the week-by-week dataframes
        data_df = pd.concat(results)

        # Add the dataframe for this position to the collection
        position_data[position]=data_df
    
    # Save the raw data
    pickle.dump(position_data, open(raw_data_path, 'wb'))
    
elif download_data is False:
    position_data = pickle.load(open(raw_data_path, 'rb'))
    print('Loaded data from file', end='')

print('\n')

Loaded data from file

CPU times: user 18.8 ms, sys: 11.8 ms, total: 30.6 ms
Wall time: 30.7 ms


In [5]:
# Take a look at the result
for position, data_df in position_data.items():
    print(f'\nPosition: {position}\n')
    print(data_df.head())


Position: qb

  Rank                Name   Age   Exp  G Cmp Att   Cm%  PYd  Y/Att PTD Int  \
0    1      Josh Allen BUF  24.0   3.0  1  33  46  71.7  312   6.78   2   0   
1    2  Russell Wilson SEA  32.0   9.0  1  31  35  88.6  322   9.20   4   0   
2    3    Aaron Rodgers GB  37.0  16.0  1  32  44  72.7  364   8.27   4   0   
3    4   Lamar Jackson BAL  23.0   3.0  1  20  25  80.0  275  11.00   3   0   
4    5    Kyler Murray ARI  23.0   2.0  1  26  40  65.0  230   5.75   1   1   

  Rsh RshYd RshTD  FP/G FantPt  Year  Week Scoring profile  
0  14    57     1  32.2   32.2  2020     1               p  
1   3    29     0  31.8   31.8  2020     1               p  
2   1     2     0  30.8   30.8  2020     1               p  
3   7    45     0  27.5   27.5  2020     1               p  
4  13    91     1  26.3   26.3  2020     1               p  

Position: rb

  Rank                     Name   Age  Exp  G Rsh RshYd Y/Rsh RshTD Rec RecYd  \
0    1           Josh Jacobs LV  22.0  2.0  1  2

## 2. Fix the player name/team column

In [6]:
for position, data_df in position_data.items():
    test_df = data_df.copy()
    test_df[['Player', 'Team']] = test_df['Name'].str.extract(r'^(.*?\b(?:I{1,3}|IV)?)(?:\s+)([A-Z]{2,3})$')
    test_df.drop(columns=['Name'], inplace=True)
    position_data[position]=test_df
    print(test_df.head())

  Rank   Age   Exp  G Cmp Att   Cm%  PYd  Y/Att PTD  ... Rsh RshYd RshTD  \
0    1  24.0   3.0  1  33  46  71.7  312   6.78   2  ...  14    57     1   
1    2  32.0   9.0  1  31  35  88.6  322   9.20   4  ...   3    29     0   
2    3  37.0  16.0  1  32  44  72.7  364   8.27   4  ...   1     2     0   
3    4  23.0   3.0  1  20  25  80.0  275  11.00   3  ...   7    45     0   
4    5  23.0   2.0  1  26  40  65.0  230   5.75   1  ...  13    91     1   

   FP/G FantPt  Year  Week  Scoring profile          Player Team  
0  32.2   32.2  2020     1                p      Josh Allen  BUF  
1  31.8   31.8  2020     1                p  Russell Wilson  SEA  
2  30.8   30.8  2020     1                p   Aaron Rodgers   GB  
3  27.5   27.5  2020     1                p   Lamar Jackson  BAL  
4  26.3   26.3  2020     1                p    Kyler Murray  ARI  

[5 rows x 21 columns]
  Rank   Age  Exp  G Rsh RshYd Y/Rsh RshTD Rec RecYd RecTD  FP/G FantPt  Year  \
0    1  22.0  2.0  1  25    93   3.7 

## 3. Data reshaping

Now let's get to work transforming this data. We will treat each player as an independent sample within each year, this will give us a set of time series for position level performance across a season. The data structure here might be ragged, meaning that the rows are not all the same length. But, I think that is better than trying to put in zeros or placeholder values for the weeks that a player didn't play. The best data structure for this will be a Pandas dataframe with a multi-level index. The index levels will be: player, season (year) and week, and we will have a separate dataframe for each position.

In [7]:
for position, data_df in position_data.items():

    indexes=[
        data_df['Year'].tolist(),
        data_df['Week'].tolist(),
        data_df['Player'].tolist()
    ]

    index_tuples=list(zip(*indexes))
    index=pd.MultiIndex.from_tuples(index_tuples, names=['Season', 'Week', 'Player'])
    data_df.set_index(index, inplace=True)

    # Drop unnecessary columns
    data_df.drop(['Year', 'Player', 'Week'], axis=1, inplace=True)
    print(data_df.head())

                           Rank   Age   Exp  G Cmp Att   Cm%  PYd  Y/Att PTD  \
Season Week Player                                                             
2020   1    Josh Allen        1  24.0   3.0  1  33  46  71.7  312   6.78   2   
            Russell Wilson    2  32.0   9.0  1  31  35  88.6  322   9.20   4   
            Aaron Rodgers     3  37.0  16.0  1  32  44  72.7  364   8.27   4   
            Lamar Jackson     4  23.0   3.0  1  20  25  80.0  275  11.00   3   
            Kyler Murray      5  23.0   2.0  1  26  40  65.0  230   5.75   1   

                           Int Rsh RshYd RshTD  FP/G FantPt Scoring profile  \
Season Week Player                                                            
2020   1    Josh Allen       0  14    57     1  32.2   32.2               p   
            Russell Wilson   0   3    29     0  31.8   31.8               p   
            Aaron Rodgers    0   1     2     0  30.8   30.8               p   
            Lamar Jackson    0   7    45    

In [8]:
# Save the parsed data
pickle.dump(position_data, open(parsed_data_path, 'wb'))